# 🌐 Redes VPC – Control de Acceso 

En este laboratorio, aprenderás a configurar y controlar el acceso a servidores web en Google Cloud utilizando redes **VPC (Virtual Private Cloud)**, reglas de firewall con etiquetas y permisos **IAM (Identity and Access Management)**. Este enfoque es fundamental para proteger aplicaciones que manejan datos sensibles, como registros financieros o información de clientes, asegurando que solo los usuarios o sistemas autorizados puedan acceder a ellos.

💡 **Beneficio empresarial**: Implementar redes VPC y políticas de acceso basadas en el principio de menor privilegio permite a las empresas proteger datos contables críticos, garantizar la disponibilidad de servicios y cumplir con normativas de seguridad, todo mientras optimizan costos al evitar accesos innecesarios.

Para más información sobre redes VPC y IAM, consulta la [Documentación oficial de Google Cloud VPC](https://cloud.google.com/vpc/docs) y la [Documentación de Google Cloud IAM](https://cloud.google.com/iam/docs).

## 🎯 Objetivos del laboratorio
- Crear servidores web con **Nginx** en máquinas virtuales (VMs).
- Definir reglas de firewall etiquetadas para controlar el tráfico HTTP.
- Crear una cuenta de servicio con roles **IAM** específicos.
- Explorar las diferencias entre los roles **Network Admin** y **Security Admin**.

📊 **Concepto clave**: **Principio de menor privilegio**. Cada recurso en la nube debe tener solo los permisos necesarios para su función, similar a cómo en una empresa contable solo ciertos empleados pueden modificar registros financieros.

## 🧩 Tarea 1: Crear los servidores web

En esta tarea, crearás dos servidores web en máquinas virtuales (**blue** y **green**) con **Nginx** y configurarás una etiqueta de red en una de ellas para controlar el acceso.

### Paso 1: Crear el servidor blue con etiqueta de red

1. En **Google Cloud Console**, ve a **Compute Engine** > **VM instances**.
2. Haz clic en **Create Instance**.
3. Configura:
   - **Name**: `blue`
   - **Region**: Usa la región disponible (por ejemplo, `us-central1`).
   - **Zone**: Selecciona una zona dentro de la región (por ejemplo, `us-central1-a`).
4. En **Networking**, agrega la etiqueta de red:
   - **Network tags**: `web-server`
5. Haz clic en **Create**.

**Explicación**:
- La etiqueta `web-server` permite aplicar reglas de firewall específicas solo a esta VM, controlando qué tráfico puede llegar a ella.

💡 **Contexto empresarial**: La etiqueta de red es como un sello en un libro contable que indica que solo ciertos auditores pueden revisarlo, garantizando un acceso controlado.

### Paso 2: Crear el servidor green sin etiqueta

1. En **VM instances**, haz clic en **Create Instance**.
2. Configura:
   - **Name**: `green`
   - **Region** y **Zone**: Usa la misma región y zona que para `blue`.
   - No agregues ninguna etiqueta de red.
3. Haz clic en **Create**.

**Explicación**:
- Al no tener una etiqueta de red, esta VM no estará sujeta a las reglas de firewall asociadas con la etiqueta `web-server`.

### Paso 3: Instalar Nginx y personalizar la página de bienvenida

#### En la VM blue

1. En **VM instances**, haz clic en **SSH** junto a `blue` para abrir una terminal.
2. Instala **Nginx**:

In [ ]:
sudo apt-get install nginx-light -y

**Explicación del comando**:
- `sudo`: Ejecuta el comando con privilegios de administrador.
- `apt-get install nginx-light`: Instala una versión ligera del servidor web **Nginx**.
- `-y`: Acepta automáticamente la instalación.

3. Edita la página principal de **Nginx**:

In [ ]:
sudo nano /var/www/html/index.nginx-debian.html

4. Cambia la línea:
```html
<h1>Welcome to nginx!</h1>
```
por:
```html
<h1>Welcome to the blue server!</h1>
```
5. Guarda los cambios:
   - Presiona **CTRL + O**, luego **Enter** para guardar.
   - Presiona **CTRL + X** para salir.
6. Verifica el contenido:

In [ ]:
cat /var/www/html/index.nginx-debian.html

7. Cierra la sesión:

In [ ]:
exit

#### En la VM green

Repite los pasos anteriores, pero cambia el mensaje a:
```html
<h1>Welcome to the green server!</h1>
```

**Explicación**:
- Personalizar las páginas de bienvenida permite identificar qué servidor responde a las solicitudes HTTP.

💡 **Contexto empresarial**: Configurar servidores web diferenciados es como etiquetar documentos financieros para identificar claramente su origen, facilitando auditorías y seguimiento.

## 🛡️ Tarea 2: Crear la regla de firewall

En esta tarea, crearás una regla de firewall etiquetada para permitir tráfico HTTP solo a las VMs con la etiqueta `web-server`.

### Paso 1: Crear la regla de firewall allow-http-web-server

1. En **Google Cloud Console**, ve a **VPC network** > **Firewall**.
2. Haz clic en **Create Firewall Rule**.
3. Configura:
   - **Name**: `allow-http-web-server`
   - **Network**: `default`
   - **Targets**: **Specified target tags**
   - **Target tags**: `web-server`
   - **Source filter**: **IPv4 Ranges**
   - **Source IPv4 ranges**: `0.0.0.0/0` (permite todas las redes)
   - **Protocols and ports**: `tcp:80, icmp`
4. Haz clic en **Create**.

**Explicación**:
- Esta regla permite tráfico HTTP (puerto 80) e **ICMP** (ping) solo a las VMs con la etiqueta `web-server`, como `blue`.
- La VM `green`, sin la etiqueta, no será accesible externamente por HTTP.

💡 **Contexto empresarial**: Esta regla es como permitir que solo ciertos documentos financieros (etiquetados) sean accesibles desde el exterior, protegiendo otros registros sensibles.

### Paso 2: Crear una máquina de prueba test-vm

Ejecuta el siguiente comando en **Cloud Shell**:

In [ ]:
gcloud compute instances create test-vm --machine-type=e2-micro --subnet=default --zone=ZONE

**Explicación del comando**:
- `gcloud compute instances create`: Crea una nueva VM.
- `test-vm`: Nombre de la instancia.
- `--machine-type=e2-micro`: Tipo de VM económica.
- `--subnet=default`: Conecta la VM a la subred por defecto.
- `--zone=ZONE`: Especifica la zona (por ejemplo, `us-central1-a`).

💡 **Contexto empresarial**: La VM `test-vm` es como una estación de auditoría que permite verificar el acceso a los sistemas financieros.

### Paso 3: Probar conectividad HTTP desde test-vm

1. En **Compute Engine** > **VM instances**, copia las **IP internas** y **externas** de `blue` y `green`.
2. Abre una terminal en `test-vm` haciendo clic en **SSH**.
3. Ejecuta los siguientes comandos, reemplazando `IP-interna-blue`, `IP-interna-green`, `IP-externa-blue` y `IP-externa-green` con las direcciones correspondientes:

In [ ]:
curl IP-interna-blue
curl IP-interna-green
curl IP-externa-blue
curl IP-externa-green

**Resultados esperados**:
- ✅ `curl IP-interna-blue`: Responde con `Welcome to the blue server!`.
- ✅ `curl IP-interna-green`: Responde con `Welcome to the green server!`.
- ✅ `curl IP-externa-blue`: Responde con `Welcome to the blue server!`.
- ⛔ `curl IP-externa-green`: No responde, ya que la regla de firewall bloquea el tráfico HTTP externo a `green` (sin la etiqueta `web-server`).

**Explicación**:
- La regla de firewall permite tráfico HTTP externo solo a VMs con la etiqueta `web-server`, confirmando que el acceso está correctamente controlado.

💡 **Contexto empresarial**: Esto asegura que solo los sistemas financieros autorizados (etiquetados) sean accesibles desde el exterior, protegiendo datos sensibles en sistemas no autorizados.

## 🔐 Tarea 3: Explorar los roles IAM (Network Admin y Security Admin)

En esta tarea, explorarás cómo los roles **IAM** controlan los permisos para gestionar reglas de firewall, siguiendo el principio de menor privilegio.

📊 **Analogía contable**: Los roles **IAM** son como los niveles de autorización en una empresa contable, donde algunos empleados pueden ver reportes, otros modificarlos y solo unos pocos crear nuevas estructuras.

### Paso 1: Verificar permisos actuales en test-vm

1. En la terminal **SSH** de `test-vm`, ejecuta:

In [ ]:
gcloud compute firewall-rules list

**Resultado esperado**:
```
ERROR: (gcloud.compute.firewall-rules.list) Some requests did not succeed:
 - Insufficient Permission
```

2. Intenta eliminar la regla de firewall:

In [ ]:
gcloud compute firewall-rules delete allow-http-web-server

**Resultado esperado**:
```
ERROR: (gcloud.compute.firewall-rules.delete) Could not fetch resource:
 - Insufficient Permission
```

**Explicación**:
- La cuenta de servicio predeterminada de `test-vm` no tiene permisos para ver ni eliminar reglas de firewall, demostrando el principio de menor privilegio.

### Paso 2: Crear cuenta de servicio Network-admin

1. En **Google Cloud Console**, ve a **IAM & Admin** > **Service Accounts**.
2. Haz clic en **Create service account**.
3. Configura:
   - **Name**: `Network-admin`
4. Haz clic en **Create and Continue**.
5. Asigna el rol:
   - **Compute Engine > Compute Network Admin**
6. Haz clic en **Continue**, luego en **Done**.
7. En la cuenta creada, haz clic en los 3 puntos > **Manage keys**.
8. Crea una nueva clave:
   - **Add Key** > **Create new key** > **JSON**.
   - Descarga el archivo y renómbralo localmente como `credentials.json`.

**Explicación**:
- La cuenta de servicio `Network-admin` tiene permisos limitados para gestionar configuraciones de red, pero no para modificar reglas de firewall.

### Paso 3: Autorizar test-vm con la cuenta de servicio

1. En la terminal **SSH** de `test-vm`, sube el archivo `credentials.json` usando el ícono de subir archivo.
2. Autoriza la cuenta de servicio:

In [ ]:
gcloud auth activate-service-account --key-file credentials.json

3. Verifica si ahora puedes listar reglas de firewall:

In [ ]:
gcloud compute firewall-rules list

**Resultado esperado**:
- ✅ La lista de reglas de firewall debe mostrarse correctamente.

4. Intenta eliminar la regla:

In [ ]:
gcloud compute firewall-rules delete allow-http-web-server

**Resultado esperado**:
```
ERROR: Required 'compute.firewall.delete' permission...
```

**Explicación**:
- La cuenta `Network-admin` puede ver reglas de firewall (permiso `compute.firewall.get`), pero no eliminarlas, ya que requiere el permiso `compute.firewall.delete`.

💡 **Contexto empresarial**: Esto es como permitir que un contador vea los registros financieros, pero no pueda modificarlos, asegurando un control estricto sobre las operaciones críticas.

### Paso 4: Actualizar rol a Security Admin

1. En **Google Cloud Console**, ve a **IAM & Admin** > **IAM**.
2. Busca la cuenta `Network-admin` y haz clic en el ícono de edición (✏️).
3. Cambia el rol a **Compute Engine > Compute Security Admin**.
4. Guarda los cambios.
5. En la terminal de `test-vm`, prueba nuevamente:

In [ ]:
gcloud compute firewall-rules list

In [ ]:
gcloud compute firewall-rules delete allow-http-web-server

**Resultados esperados**:
- ✅ `gcloud compute firewall-rules list`: Muestra la lista de reglas.
- ✅ `gcloud compute firewall-rules delete allow-http-web-server`: Elimina la regla correctamente.

**Explicación**:
- El rol **Compute Security Admin** otorga permisos adicionales, como `compute.firewall.delete`, permitiendo modificar reglas de firewall.

### Paso 5: Verificar efecto de eliminar la regla

1. En la terminal de `test-vm`, ejecuta:

In [ ]:
curl IP-externa-blue

**Resultado esperado**:
- ❌ El acceso no funciona, ya que la regla `allow-http-web-server` fue eliminada, bloqueando el tráfico HTTP externo.

**Explicación**:
- Esto demuestra cómo las reglas de firewall controlan el acceso externo, incluso si el servidor web sigue activo.

💡 **Contexto empresarial**: Eliminar la regla es como cerrar el acceso público a un sistema contable, asegurando que los datos financieros solo sean accesibles internamente o con nuevas autorizaciones.

## 📎 Apéndice: Desglose detallado de comandos

| **Comando** | **Descripción** | **Uso en contexto contable** |
|-------------|-----------------|-----------------------------|
| `sudo apt-get install nginx-light -y` | Instala el servidor web Nginx | Configura un sistema para mostrar reportes financieros en línea |
| `sudo nano /var/www/html/index.nginx-debian.html` | Edita la página web por defecto | Personaliza un informe financiero para identificar su origen |
| `cat /var/www/html/index.nginx-debian.html` | Muestra el contenido del archivo | Verifica un informe antes de su publicación |
| `gcloud compute instances create` | Crea una VM | Configura una nueva estación de auditoría |
| `curl` | Prueba la conectividad HTTP | Valida el acceso a un sistema contable en línea |
| `gcloud compute firewall-rules list` | Lista las reglas de firewall | Revisa las políticas de acceso a datos financieros |
| `gcloud compute firewall-rules delete` | Elimina una regla de firewall | Cierra el acceso público a un sistema financiero |
| `gcloud auth activate-service-account` | Autoriza una cuenta de servicio | Habilita a un sistema automatizado para realizar auditorías |

## 🚀 Resumen de conceptos

| **Concepto** | **Explicación contable simplificada** |
|--------------|--------------------------------------|
| **VPC** | Departamento contable virtual con reglas específicas |
| **Firewall tags** | Sellos que identifican qué sistemas financieros son accesibles externamente |
| **IAM roles** | Niveles de autorización para empleados o sistemas automáticos |
| **Principio de menor privilegio** | Solo otorgar permisos necesarios, como limitar quién puede modificar registros |
| **Nginx** | Servidor web para mostrar reportes financieros en línea |

💡 **Conclusión empresarial**: Configurar redes VPC, reglas de firewall etiquetadas y roles **IAM** permite a las empresas proteger datos financieros sensibles, controlar accesos y garantizar la disponibilidad de servicios críticos. Este enfoque basado en el principio de menor privilegio reduce riesgos de seguridad y facilita el cumplimiento normativo, optimizando costos al evitar accesos innecesarios.

Para más información, consulta la [Documentación de Google Cloud VPC](https://cloud.google.com/vpc/docs) y la [Documentación de Google Cloud IAM](https://cloud.google.com/iam/docs).